# IBM Qiskit Primitives v2

Sampler primitives
    - low-level execution
    - returns single shot measurement
    - circuit should include measurement

Estimator primitives
    - high-level execution
    - returns expectation values of observables
    - circuit should not include measurement

PUB - Primitive Unified Bloc
    - program input for the Sampler
    - generally a tuple

    - Circuit - ISA quantum circuit containing 1 or more ClassicalRegister and measure instructions
    - Parameter Values - tensor (ND-array) of sets of parameter values to evaluate parametric circuit

Shots - number of samples or repetitions to measure the circuit

Allowed pub-like inputs

In [1]:
# Load qiskit runtime service and select backend
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend('ibm_sherbrooke')

/var/folders/kf/vmdscp_n18qfgr6xz5zv0sw40000gn/T/ipykernel_41205/230416974.py:4: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, "ibm_cloud", "ibm_quantum_platform", and "local" will be the only valid channels. Open Plan users should migrate now.  All other users should review the migration guide (https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp)to learn when to migrate.
  service = QiskitRuntimeService()


In [2]:
# Create a circuit and transpile to a backend ISA circuit
from qiskit import QuantumCircuit, transpile

bell_meas = QuantumCircuit(2)
bell_meas.h(0)
bell_meas.cx(0, 1)
bell_meas.measure_all()

isa_bell_meas = transpile(bell_meas, backend)
pub_bell_meas = (isa_bell_meas)

In [ ]:
# Initialize a Sampler primitive and run pub
from qiskit_ibm_runtime import SamplerV2

sampler = SamplerV2(backend)
job_bell_meas = sampler.run([pub_bell_meas], shots=2)

result_bell_meas = job_bell_meas.result()
data_bell_meas = result_bell_meas[0].data
# data_bell_meas
print(data_bell_meas)
## this block of code does not stop running - neither at shots=10 nor at shots=2
## it went as long as two minutes before I stopped it

Sampler job returns `PrimitiveResult` object containing ordered list of PubResult 
PubResult = data container for a single input Pub's execution result
PubResult.data = contains measurement outcome data for classical registers in pub circuit
PubResult.metadata = metadata a primitive might record

PubResult.data = DataBin class
DataBin stores outcomes of measurements in ClassicalRegisters in the pub circuit



In [ ]:
# examples of accessing outcomes
bits = data_bell_meas.meas
print("shape: ", bits.shape)
print("num_bits: ", bits.num_bits)
print("num_shots: ", bits.num_shots)
print("array \n", bits.array)
## this block does not run because in the previous block data_bell_meas is never defined due to the long code run

## ISA Circuits

ISA circuit is a QuantumCircuit satisfying:
1. same number of qubits as device
2. only contains basic gates for a device
3. satisfies connectivity of a device

Obtained by transpilation

## Parametric circuits

* circuit that contains unbound parameter


In [ ]:
# SamplerV2 parametric example
## This is an example only because this exercise does not define para_bell_meas

import numpy as np

# parameter values to evaluate with 20 theta values
param_vals = np.linspace(0, np.pi, 20)

# transpile to an ISA circuit for the intended backend
isa_par_bell_meas = transpile(par_bell_meas, backend)

# construct pub and run
# pub and result shape
pub_par_bell_meas = (isa_par_bell_meas, param_vals)
job_par_bell_meas = sampler.run([pub_par_bell_meas], shots=1000)
result_par_bell_meas = job_par_bell_meas.result()

# extract creg data
bits = result_par_bell_meas[0].data.meas

########

import matplotlib.pyplot as plt
import seaborn
seaborn.set()

p0s = np.sum(bits.array == 0, axis=1) / bits.num_shots
p1s = np.sum(bits.array == 3, axis=1) / bits.num_shots

plt.plot(param_vals, p0s, label="P('00')")
plt.plot(param_vals, p1s, label="P('11')")
plt.xlabel("0")
plt.legend()
plt.show()

2025-06-29 - STOPPED HERE - 14:46 IN THE VIDEO - I need to cut off this exercise until later, maybe tomorrow